In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import ODE_utils as odeu
from scipy import optimize
import tables
from mpmath import *


In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
A=.115
vscalarfield=False


In [3]:
Tol=1
N=50
Nul=N
Nup=10**(40)
scal=10
umax=10
vmax=200
datatype=np.float64


#ru0=5.0
#dr0v=.4
ru0=5.0
#dr0v=0.045
dr0v=1/(4*.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989991992
r-: 0.6877501000800801
rc: 0.0
Number of points for highest iteration is 500000000,50000X10000
Total number of points is 500000000
Runtime: about 106.84 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
drnpvf=np.zeros((2,Nv),dtype=datatype)
dsignpvf=np.zeros((2,Nv),dtype=datatype)
dphinpvf=np.zeros((2,Nv),dtype=datatype)
drnpuf=np.zeros((2,Nv),dtype=datatype)
dsignpuf=np.zeros((2,Nv),dtype=datatype)
dphinpuf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
#drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
 


rnpf[0], signpf[0], phinpf[0] , drnpvf[0], dsignpvf[0], dphinpvf[0], drnpuf[0], dsignpuf[0], dphinpuf[0] = odeu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,A,datatype)
massnpf[0]=M0

print(dsignpuf[0])

Using Standard Coordinates Along V
[0.         0.00011642 0.00023232 ... 0.0385048  0.03850483 0.03850486]


In [ ]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)

massnpf[0][0]=1.0

#drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
#dsigunpvalue=100*np.exp(np.nanmax(signpf))
#drunptemp=drunp[0]
    

i=0
dv=dv0
if M0>0:
    du0=M0/Nul
elif M0==0:
    du0=1/Nul

du=du0

###fix this part (put in front of boundary conditions so it is an initial condition)
#if bdytype=="edd" or bdytype=="fulledd":
    #bdyvalue=1.0
    #dsignpuf[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
#else:
    #bdyvalue=0.0
    #dsigunp[0]=0.0
######
    
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

#drunp=np.append(drunp,np.nan)
#dsigunp=np.append(dsigunp,np.nan)
#dphinpuf=np.append(dphinpuf,np.nan)
drunptemp=np.nanmax(np.abs(drnpuf[0]))     
dsigunpvalue=np.nanmax(np.abs(dsignpuf[0]))
dphiutemp=np.nanmax(np.abs(dphinpuf[0]))

rnpf[1][0]=rnpf[0][0]

i=0
TempTol=1 
TempTolv=1
Eres=False
while max(urange)<umax and i<50000 and rnpf[1][0]>0.0 and du>0.0:

    
    du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp),abs(dphiutemp)])

    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(1*Nul)
            #print(rplus)
        else:
            du=1/Nul
    

    print("%.24f" % urange[i])
    print("%.24f" % du)
    
    ###fix this section (dphinpuf is in the wrong form)
    #if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        #u=urange[i]+du
        #phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        #dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    #else:
        #continue
        #phinpf[1][0]=0.0
        #dphinpuf[i+1]=0.0
    

    ###Predictor###
    drnpvf[1]=drnpvf[0]+du*odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q)
    dsignpvf[1]=dsignpvf[0]+du*odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)
    dphinpvf[1]=dphinpvf[0]+du*odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])
    drnpuf[1]=drnpuf[0]+du*odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])#(drnpuf[0]*dsignpuf[0]-rnpf[0]*np.power(dphinpuf[0],2.0))
    
    rnpf[1]=rnpf[0]+du*drnpuf[0]
    signpf[1]=signpf[0]+du*dsignpuf[0]
    phinpf[1]=phinpf[0]+du*dphinpuf[0]
    
    dsignpuf[1][0]=0.0#-1/(4*dsignpvf[1][0])*np.exp()
    dphinpuf[1][0]=0.0
    signpf[1][0]=0.0
    phinpf[1][0]=0.0
     
    if any(phinpf[1]) or True:
        for j in range(0,Nv-1):
            dphinpuf[1][j+1]=dphinpuf[1][j]+dv*odeu.Phifunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j])         
            #dsignpuf[1][j+1]=dsignpuf[1][j]+dv*odeu.Sigfunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j], rnpf[1][j],signpf[1][j],Q)                                                                             
    #Phifuncnp=odeu.Phifunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1])
    else:
        dphinpuf[1]=0.0
    Sigfuncnp=odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q)
    dsignpuf[1]=dsignpuf[1][0]+1/2*dv*(np.cumsum(Sigfuncnp+np.roll(Sigfuncnp,-1))-(Sigfuncnp+np.roll(Sigfuncnp,-1)))
    
    ###Correction###
    drnpvf[1]=drnpvf[0]+1/2*du*(odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q)+odeu.Rfunc(drnpvf[1],drnpuf[1],rnpf[1],signpf[1],Q))
    dsignpvf[1]=dsignpvf[0]+1/2*du*(odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)+odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q))
    dphinpvf[1]=dphinpvf[0]+1/2*du*(odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])+odeu.Phifunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1]))
    drnpuf[1]=drnpuf[0]+1/2*du*(odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])+odeu.Coneq(drnpuf[1],dsignpuf[1],dphinpuf[1],rnpf[1]))
    
    rnpf[1]=rnpf[0]+1/2*du*(drnpuf[0]+drnpuf[1])
    signpf[1]=signpf[0]+1/2*du*(dsignpuf[0]+dsignpuf[1])
    phinpf[1]=phinpf[0]+1/2*du*(dphinpuf[0]+dphinpuf[1])
    
    dsignpuf[1][0]=0.0
    dphinpuf[1][0]=0.0
    
    if np.any(phinpf[1]):   
        for j in range(0,Nv-1):
            dphinpuf[1][j+1]=dphinpuf[1][j]+1/2*dv*(odeu.Phifunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j])+odeu.Phifunc(drnpvf[1][j+1],drnpuf[1][j+1],dphinpuf[1][j+1],dphinpvf[1][j+1],rnpf[1][j+1]))         
            #dsignpuf[1][j+1]=dsignpuf[1][j]+1/2*dv*(odeu.Sigfunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j],signpf[1][j],Q)+odeu.Sigfunc(drnpvf[1][j+1],drnpuf[1][j+1],dphinpuf[1][j+1],dphinpvf[1][j+1],rnpf[1][j+1],signpf[1][j+1],Q))    
    else:
        dphinpuf[1]=0.0
    Sigfuncnp=odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q)
    dsignpuf[1]=dsignpuf[1][0]+1/2*dv*(np.cumsum(Sigfuncnp+np.roll(Sigfuncnp,-1))-(Sigfuncnp+np.roll(Sigfuncnp,-1)))
    
    
    massnpf[1]=(1+4.0*np.exp(-signpf[1])*drnpuf[1]*drnpvf[1])*rnpf[1]/2.0+np.power(Q,2.0)/(2.0*rnpf[1])
    
    drunptemp=np.nanmax(np.abs(drnpuf[1]))      
    dsigunpvalue=np.nanmax(np.abs(dsignpuf[1]))    
    dphiutemp=np.nanmax(np.abs(dphinpuf[1]))
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>1000:#0.1:
        TempTol=TempTol/2
        Eres=True
    else:
        
        if urange[i]+du<umax:
            urange=np.append(urange,urange[i]+du)
        else:
            break
        
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        #drunp=np.append(drunp,np.nan)
        #dsigunp=np.append(dsigunp,np.nan)
        #dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0]=rnpf[1]
        phinpf[0]=phinpf[1]
        signpf[0]=signpf[1]
        drnpvf[0]=drnpvf[1]
        dsignpvf[0]=dsignpvf[1]
        dphinpvf[0]=dphinpvf[1]
        drnpuf[0]=drnpuf[1]
        dsignpuf[0]=dsignpuf[1]
        dphinpuf[0]=dphinpuf[1]
        massnpf[0]=massnpf[1]
    
        
        
        if Eres==True:
            TempTol=2*TempTol
        else:
            TempTol=1
        Eres=False
        i+=1
    
    #print(TempTol)
    print(drnpuf[1][-1])
    print(dsignpuf[1][-1])
    print(massnpf[1][-1])
    #print(dphinpuf[1][-1])
    print('---')
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    #if rnpf[1][0]<0.0 or np.isnan(du):
        #break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


0.000000000000000000000000
0.020000000000000000416334
-0.5769116605632256
0.03863815308012463
1.000000004582915
---
0.020000000000000000416334
0.020000000000000000416334
-0.5773584217934615
0.03877210305151807
1.0000000087710788
---
0.040000000000000000832667
0.020000000000000000416334
-0.5778070809888602
0.03890675451960648
1.0000000130006033
---
0.059999999999999997779554
0.020000000000000000416334
-0.5782576501769306
0.03904211242874271
1.0000000172719674
---
0.080000000000000001665335
0.020000000000000000416334
-0.5787101414865014
0.039178181767263176
1.000000021585679
---
0.100000000000000005551115
0.020000000000000000416334
-0.5791645671487853
0.0393149675679591
1.0000000259422324
---
0.120000000000000009436896
0.020000000000000000416334
-0.5796209394984582
0.03945247490855894
1.0000000303421364
---
0.140000000000000013322676
0.020000000000000000416334
-0.5800792709747499
0.03959070891221326
1.0000000347859
---
0.160000000000000003330669
0.020000000000000000416334
-0.580539574122

-0.6141501153353108
0.05022431880797826
1.0000004407043492
---
1.440000000000000834887715
0.020000000000000000416334
-0.6147685539039957
0.05042342945135224
1.000000449550896
---
1.460000000000000852651283
0.020000000000000000416334
-0.6153900731927813
0.05062373628010363
1.000000458498216
---
1.480000000000000870414851
0.020000000000000000416334
-0.6160146960806551
0.05082524899292742
1.0000004675476781
---
1.500000000000000888178420
0.020000000000000000416334
-0.6166424456723606
0.051027977388037674
1.0000004767006665
---
1.520000000000000905941988
0.020000000000000000416334
-0.6172733453011753
0.05123193136440648
1.0000004859585958
---
1.540000000000000923705556
0.020000000000000000416334
-0.61790741853173
0.0514371209230246
1.0000004953228976
---
1.560000000000000941469125
0.020000000000000000416334
-0.6185446891628702
0.05164355616817903
1.000000504795027
---
1.580000000000000959232693
0.020000000000000000416334
-0.6191851812305588
0.0518512473087462
1.000000514376455
---
1.600000

-0.6672849969569927
0.06800397733756051
1.0000014247583129
---
2.860000000000002096101070
0.020000000000000000416334
-0.6681952802688922
0.06831930383186102
1.000001445898165
---
2.880000000000002113864639
0.020000000000000000416334
-0.6691110399830903
0.06863685620365557
1.000001467322113
---
2.900000000000002131628207
0.020000000000000000416334
-0.6700323251748594
0.06895665574902714
1.0000014890347182
---
2.920000000000002149391776
0.020000000000000000416334
-0.6709591855034628
0.0692787240229855
1.0000015110406097
---
2.940000000000002167155344
0.020000000000000000416334
-0.67189167122082
0.06960308284330584
1.0000015333444996
---
2.960000000000002184918912
0.020000000000000000416334
-0.6728298331803272
0.06992975429443914
1.000001555951203
---
2.980000000000002202682481
0.020000000000000000416334
-0.673773722845834
0.07025876073148545
1.0000015788655998
---
3.000000000000002220446049
0.020000000000000000416334
-0.674723392300782
0.07059012478424355
1.0000016020927085
---
3.0200000

-0.7534662551759862
0.09910609357286253
1.0000041879046933
---
4.339999999999995416999354
0.020000000000000000416334
-0.7549654226682817
0.09966627482192227
1.000004250864682
---
4.359999999999994990673713
0.020000000000000000416334
-0.7564760849610153
0.10023131391543819
1.0000043148655224
---
4.379999999999994564348071
0.020000000000000000416334
-0.7579983731652646
0.1008012683736567
1.0000043799277307
---
4.399999999999994138022430
0.020000000000000000416334
-0.7595324203769115
0.1013761965884568
1.000004446072297
---
4.419999999999993711696789
0.020000000000000000416334
-0.7610783617141117
0.10195615783954874
1.0000045133206859
---
4.439999999999993285371147
0.020000000000000000416334
-0.7626363343556134
0.10254121231106272
1.000004581694863
---
4.459999999999992859045506
0.020000000000000000416334
-0.7642064775799462
0.10313142110848114
1.0000046512172973
---
4.479999999999992432719864
0.020000000000000000416334
-0.7657889328055034
0.10372684627596686
1.00000472191099
---
4.499999

-0.9041845008956129
0.1576167538890287
1.0000135822610012
---
5.799999999999964295227528
0.020000000000000000416334
-0.9070496641838333
0.15876218251805627
1.0000138263582803
---
5.819999999999963868901887
0.020000000000000000416334
-0.9099448694655424
0.15992056657940962
1.0000140756918652
---
5.839999999999963442576245
0.020000000000000000416334
-0.9128705846227351
0.16109210886881462
1.000014330396287
---
5.859999999999963016250604
0.020000000000000000416334
-0.9158272872081513
0.16227701629412775
1.0000145906100673
---
5.879999999999962589924962
0.020000000000000000416334
-0.9188154646947593
0.1634754999787369
1.000014856475937
---
5.899999999999962163599321
0.020000000000000000416334
-0.9218356147329657
0.16468777536812992
1.0000151281409357
---
5.919999999999961737273679
0.020000000000000000416334
-0.9248882454158339
0.16591406233963896
1.0000154057565873
---
5.939999999999961310948038
0.020000000000000000416334
-0.9279738755526011
0.1671545853156087
1.0000156894790482
---
5.9599

...

-1.2755363500528414e-16
-460877116.08606464
1.006531332426676
---
9.833978158119409229698249
0.000000000043395515424691
-1.2503331855596088e-16
-458931798.38123757
1.0065313327978749
---
9.833978158162805627284797
0.000000000043579460108332
-1.225628007713882e-16
-456994675.0956116
1.0065313331690742
---
9.833978158206384989625803
0.000000000043764186083384
-1.2014109768415015e-16
-455065711.78529835
1.006531333540274
---
9.833978158250149093078107
0.000000000043949696674655
-1.177672447689738e-16
-453144874.1508715
1.0065313339114743
---
9.833978158294097937641709
0.000000000044135995221125
-1.15440296558573e-16
-451232128.0367341
1.006531334282675
---
9.833978158338233299673448
0.000000000044323085076007
-1.131593262670827e-16
-449327439.4305453
1.0065313346538762
---
9.833978158382556955530163
0.000000000044510969606813
-1.109234254209336e-16
-447430774.462598
1.006531335025078
---
9.833978158427067128855015
0.000000000044699652195408
-1.0873170349702054e-16
-445542099.4052412
1.006

-3.154986580587966e-17
-342734390.1474893
1.0065313584117406
---
9.833978161647323190663883
0.000000000058354225823074
-3.09264767046585e-17
-341286882.059735
1.0065313587829725
---
9.833978161705678289195021
0.000000000058601724974883
-3.031540506674614e-17
-339845475.2745221
1.0065313591542049
---
9.833978161764280301326835
0.000000000058850275949222
-2.971640751278672e-17
-338410144.12890714
1.0065313595254382
---
9.833978161823131003416165
0.000000000059099883224486
-2.912924547233374e-17
-336980863.0676399
1.0065313598966714
---
9.833978161882230395463012
0.000000000059350551298177
-2.855368508883117e-17
-335557606.64272654
1.0065313602679056
---
9.833978161941580253824213
0.000000000059602284686976
-2.798949712647208e-17
-334140349.51296943
1.00653136063914
---
9.833978162001182354856610
0.000000000059855087926829
-2.7436456878897585e-17
-332729066.44351965
1.0065313610103748
---
9.833978162061036698560201
0.000000000060108965573030
-2.689434407969988e-17
-331323732.3054449
1.006

-7.350244364254775e-18
-251615149.58594924
1.0065313855129276
---
9.833978166628204675703273
0.000000000079486469844568
-7.205012167088941e-18
-250551829.0907555
1.0065313858841942
---
9.833978166707691315195916
0.000000000079823803612128
-7.06264959007775e-18
-249492993.4310831
1.0065313862554617
---
9.833978166787515462488045
0.000000000080162571801939
-6.923099932797897e-18
-248438623.73014748
1.0065313866267291
---
9.833978166867678893936500
0.000000000080502780524673
-6.786307615162805e-18
-247388701.19043905
1.0065313869979973
---
9.833978166948181609541280
0.000000000080844435917080
-6.652218155286015e-18
-246343207.09339613
1.0065313873692658
---
9.833978167029025385659224
0.000000000081187544142094
-6.520778147781954e-18
-245302122.79907322
1.006531387740535
---
9.833978167110213775004013
0.000000000081532111388950
-6.391935242495479e-18
-244265429.74580896
1.0065313881118045
---
9.833978167191745001218806
0.000000000081878143873297
-6.265638123651683e-18
-243233109.44989476
1

-1.678571348873737e-18
-183905420.43778977
1.006531412987948
---
9.833978173521806809276313
0.000000000108751552577350
-1.6454047926524433e-18
-183127769.81504694
1.0065314133592498
---
9.833978173630558927698075
0.000000000109213365183223
-1.6128935683522546e-18
-182353401.25123715
1.006531413730552
---
9.833978173739772898898082
0.000000000109677142640433
-1.5810247273783953e-18
-181582300.92265224
1.0065314141018549
---
9.833978173849450499233171
0.000000000110142893323724
-1.5497855769854492e-18
-180814455.06368735
1.0065314144731583
---
9.833978173959593505060184
0.000000000110610625643594
-1.5191636752220675e-18
-180049849.96659142
1.006531414844462
---
9.833978174070203692735959
0.000000000111080348046450
-1.48914682597556e-18
-179288471.98122445
1.0065314152157663
---
9.833978174181284614974174
0.000000000111552069014758
-1.4597230741144032e-18
-178530307.5148194
1.006531415587071
---
9.833978174292836271774831
0.000000000112025797067200
-1.430880700726724e-18
-177775343.031731

-3.833351606578586e-19
-134391114.03083205
1.006531440465572
---
9.833978182954389524184080
0.000000000148819363127026
-3.7576092725016136e-19
-133822499.44826202
1.006531440836909
---
9.833978183103209147475354
0.000000000149451699695180
-3.683363515452687e-19
-133256286.20876211
1.0065314412082467
---
9.833978183252661153801455
0.000000000150086728131291
-3.610584764852548e-19
-132692464.19132777
1.0065314415795847
---
9.833978183402747319519221
0.000000000150724459914786
-3.53924403440186e-19
-132131023.31752041
1.0065314419509233
---
9.833978183553471197342333
0.000000000151364906574125
-3.4693129105365094e-19
-131571953.55128942
1.0065314423222622
---
9.833978183704836339984467
0.000000000152008079687010
-3.400763541111023e-19
-131015244.89879389
1.0065314426936018
---
9.833978183856844523802465
0.000000000152653990880600
-3.3335686243055843e-19
-130460887.40822689
1.0065314430649417
---
9.833978184009499301510004
0.000000000153302651831715
-3.267701397752238e-19
-129908871.169638

In [ ]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [ ]:
Nu=i+1

print(Nu)
print(len(urange))
#print(len(rnp))
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

In [ ]:
#print(rnpf[1,0])

In [ ]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [ ]:
#print(rnpf)

In [ ]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [ ]:
switch=0.0
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD,switch]))
np.save('urange',urange)
np.save('vrange',vrange)

In [ ]:
stop = timeit.default_timer()

In [ ]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            